In [1]:
import pandas as pd

# coding: utf-8

# 만든 기능 임포트
from crawling import UrlCrawling, ContentCrawling
from preprocessing import Preprocessing
from clustering import Clustering
from summary import Summary
from database import insert, select

In [ ]:
# 링크 크롤링하는 객체 생성
url_crawler = UrlCrawling()

six_url = url_crawler.getSixUrl()                          # 6개 카테고리 url
entertainment_url = url_crawler.getEntertainmentUrl()      # 연예 url
sports_url = url_crawler.getSportsUrl()                    # 스포츠 url
all_url = six_url + entertainment_url + sports_url        # 전체 url
category = url_crawler.category                            # 카테고리 리스트

# 본문 크롤링하는 객체 생성
content_crawler = ContentCrawling([], [], [], [])

content_crawler.getSixContent(six_url)
content_crawler.getEntertainmentContent(entertainment_url)
content_crawler.getSportsContent(sports_url)

article_df = content_crawler.makeDataFrame(all_url, category)                   # 본문 데이터프레임 생성

article_df = Preprocessing.getNouns(article_df)                                 # 명사 추출

Preprocessing.removeEnglishArticle(article_df)                                  # 영어 기사 삭제

# vector_list = Preprocessing.getVector(article_df)                               # 명사 벡터화

# Clustering.addClusterNumber(article_df, vector_list)                            # 군집 번호 열 생성
# cluster_counts_df = Clustering.getClusteredArticle(article_df)                  # 군집 개수 카운트한 df

# summary_article = Summary.getSummaryArticle(article_df, cluster_counts_df)              # 요약한 기사 데이터 프레임 반환
# summary_article = Preprocessing.convertCategory(summary_article)

In [3]:
# csv로 저장
article_df.to_csv("test.csv",index=False, encoding="utf-8-sig")

In [7]:
df = pd.read_csv("test.csv")
df

,category,date,title,content,img,url,nouns
0,정치,2023.12.01. 오후 1:44,"번번이 수 싸움 밀린 野, 이재명 “사퇴 전혀 예상 못해”",두 차례 이동관 탄핵 시도 무위로윤석열 대통령이 1일 이동관 방송통신위원장이 제출한...,https://imgnews.pstatic.net/image/023/2023/12/...,https://n.news.naver.com/mnews/article/023/000...,"['두', '차례', '이동관', '탄핵', '시도', '무위', '윤석열', '대..."
1,정치,2023.12.01. 오후 1:42,"안철수 ""김기현·장제원 희생해야…한동훈·원희룡 서울 勝算지역 출마를""","인요한 '희생 혁신안건 안 받을 거면 공천관리위원장 추천해달라' 요청에 공감安 ""印...",https://imgnews.pstatic.net/image/029/2023/12/...,https://n.news.naver.com/mnews/article/029/000...,"['인', '요한', '희생', '혁신', '안건', '안', '거', '공천', ..."
2,정치,2023.12.01. 오후 1:40,與 필리버스터 취소 이은 이동관 사퇴…탄핵안 무산에 맥 빠진 野,‘언론탄압’ 멈추라며 사퇴하라더니이번엔 윤 대통령에 “사표 수리 말라”대여 수 싸움...,https://imgnews.pstatic.net/image/022/2023/12/...,https://n.news.naver.com/mnews/article/022/000...,"['언론', '탄압', '사퇴', '더니', '이번', '윤', '대통령', '사표..."
3,정치,2023.12.01. 오후 1:36,‘탈북자 강제북송 중단 결의안’ 기권한 백혜련 “투표기 오류”,백혜련 더불어민주당 의원이 중국의 탈북자 강제북송 중단 결의안에 자신이 기권한 것이...,https://imgnews.pstatic.net/image/023/2023/12/...,https://n.news.naver.com/mnews/article/023/000...,"['백혜련', '민주당', '의원', '중국', '탈북자', '강제', '북송', ..."
4,정치,2023.12.01. 오후 1:35,엑스포 실패에 입 연 이재명 “장난하듯 접근해…참 나쁜 짓”,"이재명 더불어민주당 대표는 1일 부산 엑스포 유치 실패에 대해 ""문제는 국민을 결과...",https://imgnews.pstatic.net/image/088/2023/12/...,https://n.news.naver.com/mnews/article/088/000...,"['이재명', '민주당', '대표', '부산', '엑스포', '유치', '실패', ..."
...,...,...,...,...,...,...,...
1539,스포츠,2023.12.01. 오전 12:07,"""강원대회, 청소년 넓은 세상 도약 기회… 큰 이상 품길""",인제중 댄스팀·이진영 양 축하공연동계 스포츠 프로그램·OX퀴즈 인기'2024 강원동...,https://imgnews.pstatic.net/image/654/2023/12/...,https://sports.news.naver.com/news?oid=654&aid...,"['인제', '댄스', '팀', '이진영', '양', '축하공연', '동계', '스..."
1540,스포츠,2023.12.01. 오전 12:06,"대회전 우승→회전 우승…'간판' 정동현, FIS컵 알파인 스키 연속 우승",(MHN스포츠 ) 지난 28일 남자 대회전 경기에서 우승을 차지한 한국 알파인 스키...,https://imgnews.pstatic.net/image/445/2023/12/...,https://sports.news.naver.com/news?oid=445&aid...,"['스포츠', '지난', '남자', '회전', '경기', '우승', '차지', '한..."
1541,스포츠,2023.12.01. 오전 12:02,경쟁률 8대1…박인비 IOC 선수위원 후보 32명에 선정,이제부터 진짜 레이스다. 국제올림픽위원회(IOC) 선수위원으로 거...,https://imgnews.pstatic.net/image/025/2023/12/...,https://sports.news.naver.com/news?oid=025&aid...,"['이제', '진짜', '레이스', '국제올림픽위원회', '선수', '위원', '거..."
1542,스포츠,2023.12.01. 오전 12:02,"[삼성화재배 AI와 함께하는 바둑 해설] 이창호, 본선 진출",〈예선 결승〉 ○ 김종수 9단 ● 이창호 9단 장면⑧=흑▲의 절단에 백△의...,https://imgnews.pstatic.net/image/025/2023/12/...,https://sports.news.naver.com/news?oid=025&aid...,"['예선', '결승', '김종수', '단', '이창호', '단', '장면', '의'..."


In [8]:
insert(summary_article.values.tolist())